In [1]:
import keras
import numpy as np
import pandas as pd
import matchzoo as mz

Using TensorFlow backend.
Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt


In [2]:
train_pack = mz.datasets.wiki_qa.load_data('train', task='ranking')
valid_pack = mz.datasets.wiki_qa.load_data('dev', task='ranking', filter=True)
predict_pack = mz.datasets.wiki_qa.load_data('test', task='ranking', filter=True)

In [3]:
preprocessor = mz.preprocessors.DSSMPreprocessor()
train_pack_processed = preprocessor.fit_transform(train_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 2118/2118 [00:00<00:00, 4532.53it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 18841/18841 [00:03<00:00, 4924.66it/s]
Building VocabularyUnit from a datapack.: 100%|██████████| 1614976/1614976 [00:00<00:00, 4729943.67it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit => SumRepresentUnit: 100%|██████████| 2118/2118 [00:00<00:00, 2854.96it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit => SumRepresentUnit: 100%|██████████| 18841/18841 [00:26<00:00, 704.88it/s]


In [4]:
valid_pack_processed = preprocessor.transform(valid_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit => SumRepresentUnit: 100%|██████████| 122/122 [00:00<00:00, 2091.73it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit => SumRepresentUnit: 100%|██████████| 1115/1115 [00:01<00:00, 674.54it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit => SumRepresentUnit: 100%|██████████| 237/237 [00:00<00:00, 2848.70it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit => SumRepresentUnit: 100%|██████████| 2300/2300 [00:03<00:00, 673.29it/s]


In [5]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [6]:
model = mz.models.DRMMTKSModel()
model.params['input_shapes'] = preprocessor.context['input_shapes']
model.params['task'] = ranking_task
model.guess_and_fill_missing_params()
model.build()
model.compile()
model.backend.summary()

Parameter "name" set to DRMMTKSModel.
Parameter "embedding_input_dim" set to 300.
Parameter "embedding_output_dim" set to 300.
Parameter "embedding_trainable" set to True.
Parameter "mlp_num_layers" set to 3.
Parameter "mlp_num_units" set to 64.
Parameter "mlp_num_fan_out" set to 32.
Parameter "mlp_activation_func" set to relu.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 9644)         0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 9644)         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 9644, 300)    90000       text_left[0

In [8]:
pred_x, pred_y = predict_pack_processed[:].unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, batch_size=len(pred_x))

In [9]:
train_generator = mz.DataGenerator(train_pack_processed, batch_size=64, shuffle=True)
len(train_generator)

319

In [ ]:
history = model.fit_generator(train_generator, epochs=30, callbacks=[evaluate], workers=5, use_multiprocessing=False)

Epoch 1/30


/home/bo/Documents/open-source/MatchZoo/matchzoo/data_pack/data_pack.py:167: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  relation = self._relation.loc[index].reset_index(drop=True)
